optunaでtuningしたlgbmsをRFでensemble

In [9]:
import seaborn as sns
import csv
import optuna
import itertools, logging, joblib

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
NOTE_NAME = "lgbm2"

In [2]:
ROUND = "237"
TOURNAMENT_NAME = "kazutsugi"
TARGET_NAME = f"target_{TOURNAMENT_NAME}"
PREDICTION_NAME = f"prediction_{TOURNAMENT_NAME}"
N_FEATURE = 310

In [3]:
# Load Dataset
def read_csv(file_path):
    with open(file_path, 'r') as f:
        column_names = next(csv.reader(f))
    dtypes = {x: np.float16 for x in column_names if x.startswith(('feature', 'target'))}
    df = pd.read_csv(file_path, dtype=dtypes, index_col=0)
    return df

train = read_csv("../../dataset/numerai_dataset_237/numerai_training_data.csv")
tournament = read_csv("../../dataset/numerai_dataset_237/numerai_tournament_data.csv")

# reform
train.sort_index(axis=1, inplace=True)
tournament.sort_index(axis=1, inplace=True)
cols_feature = train.columns[train.columns.str.contains("feature_")]
cols_target  = train.columns[train.columns.str.contains("target_")]

In [5]:
train.head(1).append(tournament.tail(1))

,data_type,era,feature_charisma1,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma2,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,feature_charisma27,feature_charisma28,feature_charisma29,feature_charisma3,feature_charisma30,feature_charisma31,feature_charisma32,feature_charisma33,feature_charisma34,feature_charisma35,feature_charisma36,feature_charisma37,feature_charisma38,feature_charisma39,feature_charisma4,feature_charisma40,feature_charisma41,feature_charisma42,feature_charisma43,feature_charisma44,feature_charisma45,feature_charisma46,feature_charisma47,feature_charisma48,feature_charisma49,feature_charisma5,feature_charisma50,feature_charisma51,feature_charisma52,feature_charisma53,feature_charisma54,feature_charisma55,feature_charisma56,feature_charisma57,feature_charisma58,feature_charisma59,feature_charisma6,feature_charisma60,feature_charisma61,feature_charisma62,feature_charisma63,feature_charisma64,feature_charisma65,feature_charisma66,feature_charisma67,feature_charisma68,feature_charisma69,feature_charisma7,feature_charisma70,feature_charisma71,feature_charisma72,feature_charisma73,feature_charisma74,feature_charisma75,feature_charisma76,feature_charisma77,feature_charisma78,feature_charisma79,feature_charisma8,feature_charisma80,feature_charisma81,feature_charisma82,feature_charisma83,feature_charisma84,feature_charisma85,feature_charisma86,feature_charisma9,feature_constitution1,feature_constitution10,feature_constitution100,feature_constitution101,feature_constitution102,feature_constitution103,feature_constitution104,feature_constitution105,feature_constitution106,feature_constitution107,feature_constitution108,feature_constitution109,feature_constitution11,feature_constitution110,feature_constitution111,feature_constitution112,feature_constitution113,feature_constitution114,feature_constitution12,feature_constitution13,feature_constitution14,feature_constitution15,feature_constitution16,feature_constitution17,feature_constitution18,feature_constitution19,feature_constitution2,feature_constitution20,feature_constitution21,feature_constitution22,feature_constitution23,feature_constitution24,feature_constitution25,feature_constitution26,feature_constitution27,feature_constitution28,feature_constitution29,feature_constitution3,feature_constitution30,feature_constitution31,feature_constitution32,feature_constitution33,feature_constitution34,feature_constitution35,feature_constitution36,feature_constitution37,feature_constitution38,feature_constitution39,feature_constitution4,feature_constitution40,feature_constitution41,feature_constitution42,feature_constitution43,feature_constitution44,feature_constitution45,feature_constitution46,feature_constitution47,feature_constitution48,feature_constitution49,feature_constitution5,feature_constitution50,feature_constitution51,feature_constitution52,feature_constitution53,feature_constitution54,feature_constitution55,feature_constitution56,feature_constitution57,feature_constitution58,feature_constitution59,feature_constitution6,feature_constitution60,feature_constitution61,feature_constitution62,feature_constitution63,feature_constitution64,feature_constitution65,feature_constitution66,feature_constitution67,feature_constitution68,feature_constitution69,feature_constitution7,feature_constitution70,feature_constitution71,feature_constitution72,feature_constitution73,feature_constitution74,feature_constitution75,feature_constitution76,feature_constitution77,feature_constitution78,feature_constitution79,feature_constitution8,feature_constitution80,feature_constitution81,feature_constitution82,feature_constitution83,feature_constitution84,feature_constitution85,feature_constitution86,feature_constitution87,feature_constitution88,feature

In [20]:
# signal
signals = pd.read_csv(f"{NOTE_NAME}/lgbm2_signals_train.csv", index_col=["id"])
val_signals = pd.read_csv(f"{NOTE_NAME}/lgbm2_signals_val.csv", index_col=["id"])

In [33]:
signals.head(1)

,0,1,2,3,4,5,era,target_kazutsugi,eraNum
id,,,,,,,,,
n000315175b67977,0.480082,0.497053,0.490263,0.495587,0.501191,0.4956,era1,0.75,1


--- 
### Fit RFReg


In [29]:
# train signal data
X_train = signals.iloc[:, :6].values
y_train = signals[TARGET_NAME].values
X_val = val_signals.dropna().iloc[:, :6].values
y_val = val_signals.dropna()[TARGET_NAME].values

In [28]:
def objective(trial):
    max_depth = trial.suggest_int("max_depth", 3, 29, 2)
    min_samples_leaf = trial.suggest_uniform("min_samples_leaf", 0.01, 0.50)
    reg = RandomForestRegressor(
        criterion="mse",
        max_features=None,
        max_depth=max_depth, 
        min_samples_leaf=min_samples_leaf)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    return mse

In [31]:
# set logger
logger = logging.getLogger()  # root logger
fh = logging.FileHandler(f'{NOTE_NAME}/train3.log')
fh.setFormatter(logging.Formatter('%(asctime)s|%(message)s', '%Y-%m-%d %H:%M:%S'))
logger.addHandler(fh)

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

-----  
## 各モデルのFeatureImportance, FeatureExposure

In [5]:
from scipy.stats import spearmanr
def calcFeatureExposure(X:pd.DataFrame, pred:np.ndarray)->pd.Series:
    # pred: np.array
    exposures = [spearmanr(pred, X[col])[0] for col in X.columns]
    return pd.Series(exposures, index=X.columns)

- FeatureImportance  
- FeatureExposure